In [46]:
import pandas as pd
messages = pd.read_csv("spam.csv", encoding='ISO-8859-1')
messages.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [47]:
messages.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)

In [48]:
messages.columns = ['label','message']

In [49]:
messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [50]:
#Data Cleaning and preprocessing
import re
import nltk

In [51]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [52]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [53]:
corpus=[]
for i in range(0,len(messages)):
    #Apart from a-zA-Z,replace(Subsitute) everything with space
    review = re.sub('[^a-zA-Z]',' ',messages['message'][i])
    review = review.lower()
    review = review.split() #Because we have spaces
    review = [lemmatizer.lemmatize(word,pos='v') for word in review if word not in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

In [54]:
corpus

['go jurong point crazy available bugis n great world la e buffet cine get amore wat',
 'ok lar joke wif u oni',
 'free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate c apply',
 'u dun say early hor u c already say',
 'nah think go usf live around though',
 'freemsg hey darling week word back like fun still tb ok xxx std chgs send rcv',
 'even brother like speak treat like aid patent',
 'per request melle melle oru minnaminunginte nurungu vettam set callertune callers press copy friends callertune',
 'winner value network customer select receivea prize reward claim call claim code kl valid hours',
 'mobile months u r entitle update latest colour mobiles camera free call mobile update co free',
 'gonna home soon want talk stuff anymore tonight k cry enough today',
 'six chance win cash pound txt csh send cost p day days tsandcs apply reply hl info',
 'urgent week free membership prize jackpot txt word claim c www dbuk net lccltd pobox ldnw rw',


In [55]:
y = pd.get_dummies(messages['label'])
y = y.iloc[:,1].values

In [35]:
#Train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(corpus,y,test_size=0.2,random_state=42)

In [36]:
#Creaate Bag Of Words Model
from sklearn.feature_extraction.text import CountVectorizer
#For Binary BOW, enable binary=True
cv = CountVectorizer(max_features=2500,ngram_range=(1,2))

In [37]:
X_train = cv.fit_transform(X_train).toarray()
X_test = cv.transform(X_test).toarray()

In [38]:
cv.get_feature_names_out().shape

(2500,)

In [39]:
X_train.shape , y_train.shape

((4457, 2500), (4457,))

In [40]:
cv.vocabulary_

{'still': 1971,
 'moms': 1325,
 'check': 324,
 'yo': 2491,
 'half': 889,
 'bank': 141,
 'issue': 1037,
 'strong': 1997,
 'buy': 228,
 'pick': 1565,
 'per': 1542,
 'give': 804,
 'second': 1800,
 'chance': 311,
 'play': 1577,
 'lt': 1205,
 'gt': 862,
 'lt gt': 1207,
 'private': 1641,
 'account': 9,
 'statement': 1964,
 'show': 1864,
 'un': 2216,
 'redeem': 1709,
 'point': 1607,
 'call': 239,
 'identifier': 993,
 'code': 359,
 'expire': 641,
 'private account': 1642,
 'account statement': 10,
 'statement show': 1965,
 'show un': 1866,
 'un redeem': 2217,
 'redeem point': 1710,
 'point call': 1608,
 'call identifier': 248,
 'identifier code': 994,
 'code expire': 360,
 'say': 1783,
 'never': 1401,
 'answer': 65,
 'texts': 2080,
 'confirm': 401,
 'lose': 1188,
 'pound': 1621,
 'help': 918,
 'bite': 178,
 'sort': 1932,
 'stuff': 2000,
 'come': 379,
 'people': 1540,
 'wah': 2311,
 'lucky': 1212,
 'man': 1231,
 'save': 1781,
 'money': 1328,
 'hee': 914,
 'much': 1365,
 'better': 166,
 'thank':

In [41]:
from sklearn.naive_bayes import MultinomialNB

In [42]:
spam_detect_model = MultinomialNB().fit(X_train,y_train)

In [43]:
spam_detect_model

MultinomialNB()

In [44]:
y_pred = spam_detect_model.predict(X_test)

In [45]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
print(accuracy_score(y_pred,y_test))
print(classification_report(y_pred,y_test))

0.979372197309417
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       972
           1       0.90      0.94      0.92       143

    accuracy                           0.98      1115
   macro avg       0.95      0.96      0.95      1115
weighted avg       0.98      0.98      0.98      1115



In [61]:
# Using TF-IDF model 

In [56]:
#Train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(corpus,y,test_size=0.2,random_state=42)

In [57]:
#Creating TF-IDF model
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features=2500,ngram_range=(1,2))
X_train = tv.fit_transform(X_train).toarray()
X_test = tv.transform(X_test).toarray()

In [58]:
from sklearn.naive_bayes import MultinomialNB
spam_tfidf_model = MultinomialNB().fit(X_train,y_train)

In [59]:
y_pred = spam_tfidf_model.predict(X_test)

In [60]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
print(accuracy_score(y_pred,y_test))
print(classification_report(y_pred,y_test))

0.9730941704035875
              precision    recall  f1-score   support

           0       1.00      0.97      0.98       995
           1       0.80      1.00      0.89       120

    accuracy                           0.97      1115
   macro avg       0.90      0.98      0.94      1115
weighted avg       0.98      0.97      0.97      1115

